In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark

1. ### create new spark session

In [ ]:
sc.stop()

In [4]:
from pyspark import SparkConf, SparkContext
config = SparkConf().setAppName("PySparkSession").setMaster("local[4]")
sc = SparkContext(conf=config)

In [ ]:
# Spark Session ()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pySparkSession').getOrCreate()

In [6]:
!hdfs dfs -mkdir /flights

mkdir: `/flights': File exists


In [7]:
!hdfs dfs -put /home/hadoop/Downloads/raw_flight_data.csv /flights

put: `/flights/raw_flight_data.csv': File exists


In [8]:
flights_df = spark.read.csv('/flights/raw_flight_data.csv',inferSchema=True, header=True)

In [9]:
flights_df.head(3)

[Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=11433, DestAirportID=13303, DepDelay=-3, ArrDelay=1),
 Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=14869, DestAirportID=12478, DepDelay=0, ArrDelay=-8),
 Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=14057, DestAirportID=14869, DepDelay=-4, ArrDelay=-15)]

In [10]:
flights_df.select('Carrier').distinct().show()

+-------+
|Carrier|
+-------+
|     UA|
|     AA|
|     EV|
|     B6|
|     DL|
|     OO|
|     F9|
|     YV|
|     US|
|     MQ|
|     HA|
|     AS|
|     FL|
|     VX|
|     WN|
|     9E|
+-------+



###2.count - Aggregate

In [11]:
flights_df.select('Carrier').distinct().count()

16

In [12]:
###3.Where()- just like filter method of RDD with boolean conditions statements

In [13]:
flights_df.columns

['DayofMonth',
 'DayOfWeek',
 'Carrier',
 'OriginAirportID',
 'DestAirportID',
 'DepDelay',
 'ArrDelay']

In [14]:
flights_df.where((flights_df.DepDelay>0)&(flights_df.ArrDelay>0)).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|        10397|     323|     322|
|        19|        5|     DL|          11433|        11298|      22|      41|
|        19|        5|     DL|          11298|        11433|      40|      20|
|        19|        5|     DL|          10397|        12451|      71|      75|
|        19|        5|     DL|          12451|        10397|      75|      57|
|        19|        5|     DL|          10397|        14771|      31|      38|
|        19|        5|     DL|          13204|      

In [15]:
flights_df.where((flights_df.DepDelay>0)&(flights_df.ArrDelay>0)).count()

815851

### 4.Filter()

In [16]:
flights_df.filter((flights_df.DepDelay>0)&(flights_df.ArrDelay>0)).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|        10397|     323|     322|
|        19|        5|     DL|          11433|        11298|      22|      41|
|        19|        5|     DL|          11298|        11433|      40|      20|
|        19|        5|     DL|          10397|        12451|      71|      75|
|        19|        5|     DL|          12451|        10397|      75|      57|
|        19|        5|     DL|          10397|        14771|      31|      38|
|        19|        5|     DL|          13204|      

### 5.isin()

In [17]:
flights_df.where(flights_df.Carrier.isin('DL','B6','UA')).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

###################

## Join

In [18]:
airports_df = spark.read.csv("file:///home/hadoop/Downloads/airports.csv",header=True, inferSchema=True)

In [19]:
airports_df.show()

+----------+-----------+-----+--------------------+
|airport_id|       city|state|                name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
|     10304|      Aniak|   AK|       Aniak Airport|
|     10754|     Barrow|   AK|Wiley Post/Will R...|
|     10551|     Bethel|   AK|      Bethel Airport|
|     10926|    Cordova|   AK|Merle K Mudhole S...|
|     14709|  Deadhorse|   AK|   Deadhorse Airport|
|     11336| Dillingham|   AK|  Dillingham Airport|
|     11630|  Fairbanks|   AK|Fairbanks Interna...|
|     11997|   Gustavus|   AK|    Gustavus Airport|
|     12523|     Juneau|   AK|Juneau International|
|     12819|  Ketchikan|   AK|Ketchikan Interna...|
|     10245|King Salmon|   AK| King Salmon Airport|
|     10170|     Kodiak|   AK|      Kodiak Airport|
|     13970|   Kotzebue|   AK| Ralph Wien Memorial|
|     13873|       Nome|   AK|        Nome Airport|
|     14256|

In [20]:
air_flight = flights_df.join(airports_df.select(['name','airport_id']), flights_df.OriginAirportID == airports_df.airport_id)

air_flight.show()

+----------+---------+-------+---------------+-------------+--------+--------+--------------------+----------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|                name|airport_id|
+----------+---------+-------+---------------+-------------+--------+--------+--------------------+----------+
|        19|        5|     DL|          11433|        13303|      -3|       1|Detroit Metro Way...|     11433|
|        19|        5|     DL|          14869|        12478|       0|      -8|Salt Lake City In...|     14869|
|        19|        5|     DL|          14057|        14869|      -4|     -15|Portland Internat...|     14057|
|        19|        5|     DL|          15016|        11433|      28|      24|Lambert-St. Louis...|     15016|
|        19|        5|     DL|          11193|        12892|      -6|     -11|Cincinnati/Northe...|     11193|
|        19|        5|     DL|          10397|        15016|      -1|     -19|Hartsfield-Jackso...|     10397|
|

### is null

In [21]:
from pyspark.sql.functions import *

In [22]:
flights_df.select([count(when(isnull(col),col)).alias(col) for col in flights_df.columns]).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|         0|        0|      0|              0|            0|   27444|   29033|
+----------+---------+-------+---------------+-------------+--------+--------+



### flights_df.toPandas() - Type casting spark DF to pandas DF

In [23]:
flights_df1 = flights_df.dropDuplicates()

In [24]:
def perc(flights_df1,flights_df):
    count1 = flights_df1.count()
    count2 = flights_df.count()
    res = (count2-count1)/count2*100
    return res

In [25]:
perc(flights_df1,flights_df)

0.8249927006440348

### 10.fillna() : 
    *to replace or fill missing columns with central tendancy (mean, median, mode)*
        *Here will replace missing values by 0s*


In [26]:
fill_null_df = air_flight.fillna(0,subset = ['DepDelay','ArrDelay'])

In [27]:
fill_null_df.show()

+----------+---------+-------+---------------+-------------+--------+--------+--------------------+----------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|                name|airport_id|
+----------+---------+-------+---------------+-------------+--------+--------+--------------------+----------+
|        19|        5|     DL|          11433|        13303|      -3|       1|Detroit Metro Way...|     11433|
|        19|        5|     DL|          14869|        12478|       0|      -8|Salt Lake City In...|     14869|
|        19|        5|     DL|          14057|        14869|      -4|     -15|Portland Internat...|     14057|
|        19|        5|     DL|          15016|        11433|      28|      24|Lambert-St. Louis...|     15016|
|        19|        5|     DL|          11193|        12892|      -6|     -11|Cincinnati/Northe...|     11193|
|        19|        5|     DL|          10397|        15016|      -1|     -19|Hartsfield-Jackso...|     10397|
|

In [28]:
air_flight.select(round(mean('DepDelay')).alias('mean')).show()

+----+
|mean|
+----+
|11.0|
+----+



In [29]:
from pyspark.sql.functions import stddev
flights_df.select(round(stddev('DepDelay')).alias('std_env')).show()

+-------+
|std_env|
+-------+
|   36.0|
+-------+



In [30]:
flights_df.select(round(stddev('DepDelay')).alias('std_env')).collect()[0]['std_env']

36.0

### 11.GroupBy() and agg

In [31]:
flights_df.groupBy("Carrier")

In [32]:
flights_df.groupBy("Carrier").agg(mean("DepDelay").alias("Average Delay")).show()

+-------+------------------+
|Carrier|     Average Delay|
+-------+------------------+
|     UA|12.583842334149837|
|     AA|12.064006771147696|
|     EV|14.489477751011318|
|     B6| 12.62171906310568|
|     DL| 7.362576157252043|
|     OO| 7.882683537596901|
|     F9|12.110275408919335|
|     YV| 9.544919992288413|
|     US| 4.944337992909899|
|     MQ|15.538911606849465|
|     HA|1.1510517278385919|
|     AS|0.6253646218816666|
|     FL|10.185264524505032|
|     VX|14.352406955022273|
|     WN|12.838167030213425|
|     9E| 9.734131353219817|
+-------+------------------+



In [33]:
flights_df.approxQuantile('ArrDelay',[0.5],0.001)[0]

-3.0

In [38]:
from pyspark.sql.functions import expr

median_expr = expr(f"percentile_approx({'ArrDelay'},0.5)")
flights_df.agg(median_expr.alias("median")).collect()[0][0]

-3

### 13.Spark SQL

In [39]:
#CREATE TEMPORARY TABLE USING SPARK DATA FRAME

In [40]:
flights_df.createOrReplaceTempView("flights")

In [41]:
spark.sql("select * from flights limit 10").show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

In [ ]:
#find the total number of flights of each carrier
#compute median departure delay for each carrier
#find the most busy airport

In [53]:
spark.sql("select Carrier,count(Carrier) from flights group by Carrier order by count(Carrier) desc").show()

+-------+--------------+
|Carrier|count(Carrier)|
+-------+--------------+
|     WN|        580029|
|     DL|        385040|
|     AA|        291771|
|     UA|        287601|
|     US|        235031|
|     OO|        161102|
|     EV|        158253|
|     B6|        122297|
|     MQ|        113634|
|     FL|         93013|
|     9E|         80221|
|     AS|         69056|
|     YV|         53022|
|     F9|         35821|
|     VX|         34869|
|     HA|         18658|
+-------+--------------+



In [64]:
spark.sql("select Carrier where max(count(distinct OriginAirportID) )(select )count(distinct OriginAirportID)  from flights").show()

Py4JJavaError: An error occurred while calling o726.showString.
: java.lang.UnsupportedOperationException: Cannot evaluate expression: count(distinct input[0, string, true])
	at org.apache.spark.sql.catalyst.expressions.Unevaluable$class.doGenCode(Expression.scala:261)
	at org.apache.spark.sql.catalyst.expressions.aggregate.AggregateExpression.doGenCode(interfaces.scala:87)
	at org.apache.spark.sql.catalyst.expressions.Expression$$anonfun$genCode$2.apply(Expression.scala:108)
	at org.apache.spark.sql.catalyst.expressions.Expression$$anonfun$genCode$2.apply(Expression.scala:105)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.sql.catalyst.expressions.Expression.genCode(Expression.scala:105)
	at org.apache.spark.sql.catalyst.expressions.codegen.GenerateOrdering$$anonfun$3.apply(GenerateOrdering.scala:83)
	at org.apache.spark.sql.catalyst.expressions.codegen.GenerateOrdering$$anonfun$3.apply(GenerateOrdering.scala:82)
	at scala.collection.TraversableLike$$anonfun$map$1.a

In [55]:
spark.sql("SELECT carrier, percentile_approx(DepDepart)(0.5) WITHIN GROUP (ORDER BY your_column) AS median FROM 
    your_table")

ParseException: "\nno viable alternative at input 'GROUP ('(line 1, pos 41)\n\n== SQL ==\nSELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY DepDelay) AS median FROM flights\n-----------------------------------------^^^\n"